In [3]:
#Import of libraries
import numpy as np
import pandas as pd

In [4]:
# DEBUG: Test version
from platform import python_version
print(python_version())

print(pd.__version__)

3.9.12
1.4.1


----
# 1. Define settings 
In this part, we define the variables used for controlling the application. By doing this, we are trying to make sure that we don't have to modify the rest of the code.

In [3]:
# Define which crime types should be analysed
analyse_crimetype = 'burglary'

# Define the years that should be analysed
analyse_years = [
    '2016',
    '2017',
    '2018',
    '2019',  
    '2020',
    '2021'
]

# Define for which cities the network models should be created
# TODO: This variable has currently no use
analyse_cities = [
    'denver'
]

# Define for which euclidian distances (in meters) the near-repeat networks should be created
proximity_distance = [
    200,
    300,
    400
]

# Define for which period (in days) the near-repeat networks should be created
proximity_time = [
    14
]

# Define path of both input files
INPUT_PATH_1 = '../assets/data/source/denver_1.csv'
INPUT_PATH_2 = '../assets/data/source/denver_2.csv'

----
# 2. Data import
This section is used to load the raw data into the application.

In [4]:
# Option 1: Import complete raw dataset
in1 = pd.read_csv(INPUT_PATH_1, sep=',', engine='python', encoding='utf-8')
in2 = pd.read_csv(INPUT_PATH_2, sep=',', engine='python', encoding='utf-8')
# rename lowercase column names with upper case
in2.rename(columns={'incident_id':'INCIDENT_ID', 'offense_id': 'OFFENSE_ID'}, inplace=True)
crime = pd.concat([in1, in2], ignore_index=True)

In [5]:
# Removal of duplicated ids
old = len(crime)
crime = crime.drop_duplicates(subset='INCIDENT_ID')
print(f'{old - len(crime)} duplicated ids removed')

crime.to_csv('../assets/data/source/denver.csv', index = False, header=True)

425114 duplicated ids removed


In [7]:
# Option2: Yearly data import
# TODO: append list elements instead of dataframe for performance reasons
crime_list = []

for year in analyse_years:
    path = f'../assets/data/source/denver_{year}.csv'
    pd.concat([crime, pd.read_csv(path, sep=',', engine='python', encoding='utf-8')])
print(crime.shape)

(533488, 19)


----
# 3. Data cleansing
In this section, the raw data gets processed for further use.

In [8]:
# Renaming the columns of the dataset
crime = crime.rename(columns={'INCIDENT_ID': 'id', 'OFFENSE_TYPE_ID': 'crime_type','OFFENSE_CATEGORY_ID': 'crime_cat', 
                              'DISTRICT_ID': 'district', 'PRECINCT_ID': 'precinct', 'NEIGHBORHOOD_ID': 'neighborhood', 
                              'FIRST_OCCURRENCE_DATE': 'timestamp', 'GEO_LON': 'long', 'GEO_LAT': 'lat'})
crime.head(3)

,id,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,crime_type,crime_cat,timestamp,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,long,lat,district,precinct,neighborhood,IS_CRIME,IS_TRAFFIC
0,2021224206,2021224206220200,2202,0,burglary-residence-by-force,burglary,4/18/2021 10:30:00 PM,4/19/2021 5:00:00 AM,4/21/2021 3:25:00 PM,300 W 11TH AVE,3142828.0,1692472.0,-104.992161,39.733543,6.0,611.0,civic-center,1,0
1,2021225308,2021225308240400,2404,0,theft-of-motor-vehicle,auto-theft,4/21/2021 11:25:00 PM,NaN,4/22/2021 12:01:00 AM,5700 BLK W DARTMOUTH AVE,3124936.0,1664570.0,-105.056261,39.657203,4.0,423.0,bear-valley,1,0
2,20216009452,20216009452239900,2399,0,theft-other,larceny,3/22/2021 12:51:00 PM,3/22/2021 12:51:00 PM,4/21/2021 10:13:00 PM,3412 N HUMBOLDT ST,3149191.0,1703917.0,-104.969299,39.764862,2.0,211.0,cole,1,0


In [9]:
# Only select relevant columns
df = crime[['id', 'crime_type', 'crime_cat', 'district', 'precinct', 'neighborhood', 'timestamp', 'long', 'lat']]
df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat
0,2021224206,burglary-residence-by-force,burglary,6.0,611.0,civic-center,4/18/2021 10:30:00 PM,-104.992161,39.733543
1,2021225308,theft-of-motor-vehicle,auto-theft,4.0,423.0,bear-valley,4/21/2021 11:25:00 PM,-105.056261,39.657203
2,20216009452,theft-other,larceny,2.0,211.0,cole,3/22/2021 12:51:00 PM,-104.969299,39.764862


## 3.1 Group crimes
After the data has been cleaned, the crime types must be broken down by their id.

In [10]:
df[['crime_type', 'crime_cat']].value_counts()

crime_type                     crime_cat               
traffic-accident               traffic-accident            87948
theft-of-motor-vehicle         auto-theft                  44211
traffic-accident-hit-and-run   traffic-accident            39824
theft-items-from-vehicle       theft-from-motor-vehicle    37284
traf-other                     all-other-crimes            23837
                                                           ...  
theft-of-cable-services        larceny                         1
theft-from-yards               larceny                         1
liquor-manufacturing           drug-alcohol                    1
liquor-misrepresent-age-minor  drug-alcohol                    1
riot-incite                    public-disorder                 1
Length: 199, dtype: int64

In [11]:
# Export of the crime types
crime_types = pd.DataFrame(df[['crime_type', 'crime_cat']].value_counts())
crime_types.head(3)

,,0
crime_type,crime_cat,
traffic-accident,traffic-accident,87948
theft-of-motor-vehicle,auto-theft,44211
traffic-accident-hit-and-run,traffic-accident,39824


In [12]:
# Import of the excel file with new grouping
df_group = pd.read_excel('../assets/data/source/denver_crime_types_grouped.xlsx')
df_group.head(3)

,crime_type,crime_cat,crime,crime_group
0,aggravated-assault,aggravated-assault,other,other
1,aggravated-assault-dv,aggravated-assault,other,other
2,menacing-felony-w-weap,aggravated-assault,other,other


In [13]:
# Map title of crime to its numerical value 
df = df.copy()

mapping = dict(df_group[['crime_type', 'crime']].values)
df['crime'] = df.crime_type.map(mapping)

mapping2 = dict(df_group[['crime', 'crime_group']].values)
df['crime_group'] = df.crime.map(mapping2)

df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group
0,2021224206,burglary-residence-by-force,burglary,6.0,611.0,civic-center,4/18/2021 10:30:00 PM,-104.992161,39.733543,burglary_residence,burglary_residence
1,2021225308,theft-of-motor-vehicle,auto-theft,4.0,423.0,bear-valley,4/21/2021 11:25:00 PM,-105.056261,39.657203,other,other
2,20216009452,theft-other,larceny,2.0,211.0,cole,3/22/2021 12:51:00 PM,-104.969299,39.764862,other,other


In [14]:
# Cache current state of the dataframe as a csv file
df.to_csv('../assets/data/checkpoint/01_crime_denver_until_crimes_group.csv', index = False, header=True)

## 3.2 Filter out NaN values
Filter out records that lack the information needed for further analysis

In [15]:
# Read the csv created in the previous step
df = pd.read_csv('../assets/data/checkpoint/01_crime_denver_until_crimes_group.csv', sep=',', engine='python')
df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group
0,2021224206,burglary-residence-by-force,burglary,6.0,611.0,civic-center,4/18/2021 10:30:00 PM,-104.992161,39.733543,burglary_residence,burglary_residence
1,2021225308,theft-of-motor-vehicle,auto-theft,4.0,423.0,bear-valley,4/21/2021 11:25:00 PM,-105.056261,39.657203,other,other
2,20216009452,theft-other,larceny,2.0,211.0,cole,3/22/2021 12:51:00 PM,-104.969299,39.764862,other,other


In [16]:
# Check how many records contain NaN values
df.isna().sum()

id                 0
crime_type         0
crime_cat          0
district           1
precinct           1
neighborhood       1
timestamp          0
long            4765
lat             4765
crime              0
crime_group        0
dtype: int64

In [17]:
before = len(df.index)

# Drop all records with NaN values. Crime-records which do not have a clear geographic position are not usable for the near-repeat principle.
df = df.copy()
df.dropna(inplace = True)
df.isna().sum()

after = len(df.index)
print(f'{before - after} records deleted')

4765 records deleted


## 3.3 Drop duplicated records
Filter out duplicated records in order to not distort the results

In [18]:
# Drop all duplicated records
s = df.duplicated().sum()
df.drop_duplicates(inplace = True)
e = df.duplicated().sum()

print(f'{e-s} records were removed')

0 records were removed


## 3.4 Adjust data types

In [19]:
# Check datatypes for each column
df.dtypes

id                int64
crime_type       object
crime_cat        object
district        float64
precinct        float64
neighborhood     object
timestamp        object
long            float64
lat             float64
crime            object
crime_group      object
dtype: object

In [20]:
# Change data type of district and precinct to int as they have no decimal places
df['district'] = df['district'].astype(int)
df['precinct'] = df['precinct'].astype(int)

# Change data type of timestamp to datetime format
df = df.copy()
df['timestamp'] = pd.to_datetime(df['timestamp'])

df.dtypes

id                       int64
crime_type              object
crime_cat               object
district                 int32
precinct                 int32
neighborhood            object
timestamp       datetime64[ns]
long                   float64
lat                    float64
crime                   object
crime_group             object
dtype: object

## 3.5 Date and time
Split up the timestamp into three new columns for further usage

In [21]:
# Create individual columns from timestamp
df = df.copy()

df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time

df = df.copy()

df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group,year,month,day,date,time
0,2021224206,burglary-residence-by-force,burglary,6,611,civic-center,2021-04-18 22:30:00,-104.992161,39.733543,burglary_residence,burglary_residence,2021,4,18,2021-04-18,22:30:00
1,2021225308,theft-of-motor-vehicle,auto-theft,4,423,bear-valley,2021-04-21 23:25:00,-105.056261,39.657203,other,other,2021,4,21,2021-04-21,23:25:00
2,20216009452,theft-other,larceny,2,211,cole,2021-03-22 12:51:00,-104.969299,39.764862,other,other,2021,3,22,2021-03-22,12:51:00


In [22]:
# Transform the date into the right format
df['date'] = df['date'].astype('datetime64[ns]')

In [23]:
# Create function that returns the rounded hour of the timestamp
from datetime import datetime, timedelta

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second = 0, microsecond = 0, minute = 0, hour = t.hour)
            + timedelta(hours = t.minute // 30))

In [26]:
# Create function to return name of the weekday from date
import calendar

def get_weekday(d):
    return calendar.day_name[d.weekday()]

In [29]:
# Add values to columns in dataframe
df = df.copy()
df['time_rounded'] = df['timestamp'].apply(hour_rounder)
df['hour'] = df['time_rounded'].dt.hour
df['time_rounded'] = df['time_rounded'].dt.time
df['weekday'] = df['date'].apply(get_weekday)
df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group,year,month,day,date,time,time_rounded,hour,weekday
533485,2022155867,fraud-by-telephone,white-collar-crime,5,523,gateway-green-valley-ranch,2022-03-15 03:15:00,-104.747822,39.770054,other,other,2022,3,15,2022-03-15,03:15:00,03:00:00,3,Tuesday
533486,2022155520,fraud-by-telephone,white-collar-crime,4,411,barnum-west,2022-02-14 12:00:00,-105.048253,39.723227,other,other,2022,2,14,2022-02-14,12:00:00,12:00:00,12,Monday
533487,20226005535,fraud-by-use-of-computer,white-collar-crime,3,323,hampden,2022-03-26 13:00:00,-104.878446,39.661882,other,other,2022,3,26,2022-03-26,13:00:00,13:00:00,13,Saturday


In [30]:
# Optional data split for easier exchange
for year in analyse_years:
    df_year = df.loc[(df['year'] == int(year))]
    path = f'../assets/data/source/denver_{year}.csv'
    df_year.to_csv(path, index = False, header=True)
    print(f'Done: {year}')

Done: 2016
Done: 2017
Done: 2018
Done: 2019
Done: 2020
Done: 2021


In [31]:
df.to_csv('../assets/data/checkpoint/02_crime_denver_until_data_types.csv', index = False, header=True)

----
# 4. Identification of Near-Repeats

In [32]:
# Only use records of the declared type of crime
df = df.copy()
df = df.loc[df['crime_cat'] == analyse_crimetype]
df.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group,year,month,day,date,time,time_rounded,hour,weekday
0,2021224206,burglary-residence-by-force,burglary,6,611,civic-center,2021-04-18 22:30:00,-104.992161,39.733543,burglary_residence,burglary_residence,2021,4,18,2021-04-18,22:30:00,23:00:00,23,Sunday
9,2021223562,burglary-business-by-force,burglary,3,311,speer,2021-04-21 04:17:00,-104.977886,39.725822,burglary_business,burglary_business,2021,4,21,2021-04-21,04:17:00,04:00:00,4,Wednesday
53,2021223899,burglary-business-by-force,burglary,1,113,highland,2021-04-21 00:15:00,-105.009527,39.758879,burglary_business,burglary_business,2021,4,21,2021-04-21,00:15:00,00:00:00,0,Wednesday


In [33]:
# Sort records by timestamp
df = df.sort_values(by=['timestamp'])

In [34]:
df.head(5)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,crime_group,year,month,day,date,time,time_rounded,hour,weekday
381186,20162874,burglary-business-by-force,burglary,2,222,montclair,2016-01-02 05:00:00,-104.914776,39.737851,burglary_business,burglary_business,2016,1,2,2016-01-02,05:00:00,05:00:00,5,Saturday
234824,20166879,burglary-residence-no-force,burglary,3,322,washington-virginia-vale,2016-01-02 08:35:00,-104.909504,39.691124,burglary_residence,burglary_residence,2016,1,2,2016-01-02,08:35:00,09:00:00,9,Saturday
5132,20163184,burglary-business-by-force,burglary,6,611,cbd,2016-01-02 13:38:00,-104.995367,39.741134,burglary_business,burglary_business,2016,1,2,2016-01-02,13:38:00,14:00:00,14,Saturday
93676,20164917,burglary-residence-by-force,burglary,3,322,virginia-village,2016-01-02 17:00:00,-104.904795,39.681983,burglary_residence,burglary_residence,2016,1,2,2016-01-02,17:00:00,17:00:00,17,Saturday
114356,20163610,burglary-residence-by-force,burglary,3,321,hilltop,2016-01-02 17:55:00,-104.928957,39.720677,burglary_residence,burglary_residence,2016,1,2,2016-01-02,17:55:00,18:00:00,18,Saturday


## 4.1 Euclidean distance
Define a function that can be used to determine the euclidean distance between two coordinates.

In [1]:
import math

def euclidean_distance(origin, destination):
    distance = np.sqrt((origin[0]-destination[0])**2 + (origin[1]-destination[1])**2)
    return 6371*(math.pi/180)*distance

In [6]:
# DEBUG: test method
point_a = [47.36667, 8.55]    # Zürich
point_b = [46.94809, 7.44744] # Bern

print(euclidean_distance(point_a, point_b))

131.1368574105277
131.1368574105277


## 4.2 Add columns to dataframe
The columns are used to refer to offenses that are connected as a near-repeat

In [39]:
# Create a list with column headers that will be used later
list_columns_master = []
for time in proximity_time:
    for distance in proximity_distance:
        string_column = f'day{time}_{distance}m'
        list_columns_master.append(string_column)
        list_columns_master.append(f'{string_column}_mintindex')
        list_columns_master.append(f'{string_column}_mintindex_mentions')
        list_columns_master.append(f'{string_column}_mintdate')
        list_columns_master.append(f'{string_column}_mintdatediff')

In [40]:
# Add new columns to existing dataframe
df_nr = df.copy()
df_nr = df_nr.reindex(columns = df.columns.tolist() + list_columns_master)
df_nr.head(3)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,...,day14_300m,day14_300m_mintindex,day14_300m_mintindex_mentions,day14_300m_mintdate,day14_300m_mintdatediff,day14_400m,day14_400m_mintindex,day14_400m_mintindex_mentions,day14_400m_mintdate,day14_400m_mintdatediff
381186,20162874,burglary-business-by-force,burglary,2,222,montclair,2016-01-02 05:00:00,-104.914776,39.737851,burglary_business,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234824,20166879,burglary-residence-no-force,burglary,3,322,washington-virginia-vale,2016-01-02 08:35:00,-104.909504,39.691124,burglary_residence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5132,20163184,burglary-business-by-force,burglary,6,611,cbd,2016-01-02 13:38:00,-104.995367,39.741134,burglary_business,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# DEBUG
print(f'Size df: {len(df_nr.index)} records')

Size df: 28387 records


## 4.3 Identify near repeats
Iterate over whole dataset and identify near repeats according to the temporal and geographic proximity

In [42]:
# INFO: Takes around 10 minutes for the whole dataset
import datetime
import time as tt

s = tt.time()

# Iterate over the dataframe and extraxt a tupel of the index and value for each row
for index, row in df_nr.iterrows():
    
    # Define column names for the temporary dataframe
    column_names = ['timeframe', 'type', 'distance', 'index', 'date', 'datediff']
    # Create temporary dataframe with the columns defined above
    df_nr_temp_2 = pd.DataFrame(columns = column_names)

    # Cache values from first row in variables
    index_1 = index
    id_1 = row['id']
    date = row['timestamp']
    origin_lon = row['long']
    origin_lat = row['lat']
    # Create variable that stores the number of days since
    days_last = datetime.timedelta(0)
    
    # Iterate over the list of distances for which a temporal proximity is given
    for time in proximity_time:
        
        # Create a variable with the proximity timedelta in days
        days = datetime.timedelta(time)
        # Define a variable with the delta between the time of the crime and the crime of the previous crime in the repeat (if any)
        date_ago_1 = date - days_last
        # Define a variable with the delta between the time of the crime and the temporal proximity value
        date_ago_2 = date - days

        # Extract all crime records from the dataset that were commited in the specified timeframe
        df_nr_temp = df_nr.loc[(df_nr['timestamp'] >= date_ago_2) & (df_nr['timestamp'] < date_ago_1)]
        
        # Define / reset variables for later use
        euclidean_200 = 0
        euclidean_300 = 0
        euclidean_400 = 0
  
        # Iterate over all crimes for which a temporal proximity to the initial offense is given
        for index, row in df_nr_temp.iterrows():

            # Cache each record in variables
            id_2 = row['id']
            destination_lon = row['long']
            destination_lat = row['lat']
            date_2 = row['timestamp']
            # Calculate time difference between both crimes
            date_diff = date - date_2
            # Convert to days
            date_diff = date_diff.days

            # Calculate euclidean distance based on function defined under section 4.1
            origin = [origin_lat, origin_lon]
            destination = [destination_lat, destination_lon]
            euclidean_dist = euclidean_distance(origin, destination)

            # TODO: Modularity regarding proximity settings | remove boilerplate code
            # Check if the two crimes are near each other and add a record to the temporal df if so / increase number of found nr for the matching distance
            if euclidean_dist <= .2: # up to 200m
                euclidean_200 += 1
                df_nr_temp_2 = pd.concat([df_nr_temp_2, pd.DataFrame({'timeframe': time, 'type': 'euclidean', 'distance': 200, 'index': id_2, 'date': date_2, 'datediff': date_diff}, index=['index'])], ignore_index=True)

            if euclidean_dist <= .3 and euclidean_dist > .2: # between 200m and 300m
                euclidean_300 += 1
                df_nr_temp_2 = pd.concat([df_nr_temp_2, pd.DataFrame({'timeframe': time, 'type': 'euclidean', 'distance': 300, 'index': id_2, 'date': date_2, 'datediff': date_diff}, index=['index'])], ignore_index=True)

            if euclidean_dist <= .4 and euclidean_dist > .3: # between 300m and 400m
                euclidean_400 += 1
                df_nr_temp_2 = pd.concat([df_nr_temp_2, pd.DataFrame({'timeframe': time, 'type': 'euclidean', 'distance': 400, 'index': id_2, 'date': date_2, 'datediff': date_diff}, index=['index'])], ignore_index=True)
           
        # Number count of near-repeats in the past
        df_nr.at[index_1,f'day{time}_200m'] = euclidean_200
        df_nr.at[index_1,f'day{time}_300m'] = euclidean_300
        df_nr.at[index_1,f'day{time}_400m'] = euclidean_400
        
        # Iterate over all defined distances
        for d in proximity_distance:
            # Create / empty variables
            mintdate = []
            mintdatediff = []
            mintindex = []

            # Extract records from temporal df that match regarding time and distance
            df_nr_temp_3 = df_nr_temp_2.loc[(df_nr_temp_2['timeframe'] == time) & (df_nr_temp_2['distance'] == d)]

            # Extract date of initial crime from temporal df (furthest in the past)
            mintdate = df_nr_temp_3['date'].min()
            df_nr.at[index_1,f'day{time}_{d}m_mintdate'] = mintdate
            
            # Extract date diff in days to initial crime from temporal df (highest date diff = first in series)
            mintdatediff = df_nr_temp_3['datediff'].max()
            df_nr.at[index_1,f'day{time}_{d}m_mintdatediff'] = mintdatediff  

            # Get index of initial crime from temporal df via the mintdate
            mintindex = df_nr_temp_3.loc[(df_nr_temp_3['date'] == mintdate), 'index']
            
            # Add mintindex to df if there is any & take first if there are multiple
            if len(mintindex) > 0:
                df_nr.at[index_1,f'day{time}_{d}m_mintindex'] = mintindex.iat[0]

        # TODO: Check why this is done
        days_last = days

# Write df as a csv (checkpoint)
df_nr.to_csv('../assets/data/checkpoint/03_crime_denver_until_near_repeat.csv', index = False, header=True)
e = tt.time()
print(f'done in {e-s}s')

done in 544.1256105899811s


## 4.4 Enrich data
Add additional information to the dataset for further usage

In [43]:
df_nre = df_nr.copy()

In [44]:
# INFO: Takes around 30 seconds for the whole dataset
import datetime

# Loop through all records
for index, row in df_nre.iterrows():
    
    # Cache row id
    index_1 = index
    id_1 = row['id']
    
    # Loop through headers of relevant columns (all column headers that contain 'mintindex' but not 'mentions')
    for header_mintindex in [header for header in list_columns_master if 'mintindex' in header and 'mentions' not in header]:
        
        # TODO: check if this could be removed
        subset_df = []
        column_count = 0
        
        # Get all offenses that have current row as their initial offense
        subset_df = df_nre[df_nre[f'{header_mintindex}'] == id_1]
        # Count the number of offenses for witch the current row is the initial offense
        column_count = subset_df[f'{header_mintindex}'].count()
        
        # TODO: Not sure why current record is subtracted
        if column_count > 0:
            column_count = column_count - 1 #substract current record
            
        # Add count of mentions to near-repeat df
        df_nre.at[index_1, f'{header_mintindex}_mentions'] = column_count
        
# Write df as a csv (checkpoint)
df_nre.to_csv('../assets/data/checkpoint/04_crime_denver_until_enrichment.csv', index = False, header=True) 

done


## 4.5 Reformat data and handle NaN values
Refactor data types and handle rows that contain NaN values

In [45]:
# Import of intermediate result
df = df_nre.copy()
df['date'] = df['date'].astype('datetime64[ns]')
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')

# Iterate over all date columns
for column in [s for s in list_columns_master if 'mintdate' in s and 'diff' not in s]:
    df[column] = df[column].astype('datetime64[ns]')

df.head(5)

,id,crime_type,crime_cat,district,precinct,neighborhood,timestamp,long,lat,crime,...,day14_300m,day14_300m_mintindex,day14_300m_mintindex_mentions,day14_300m_mintdate,day14_300m_mintdatediff,day14_400m,day14_400m_mintindex,day14_400m_mintindex_mentions,day14_400m_mintdate,day14_400m_mintdatediff
381186,20162874,burglary-business-by-force,burglary,2,222,montclair,2016-01-02 05:00:00,-104.914776,39.737851,burglary_business,...,0.0,NaN,0.0,NaT,NaN,0.0,NaN,0.0,NaT,NaN
234824,20166879,burglary-residence-no-force,burglary,3,322,washington-virginia-vale,2016-01-02 08:35:00,-104.909504,39.691124,burglary_residence,...,0.0,NaN,0.0,NaT,NaN,0.0,NaN,0.0,NaT,NaN
5132,20163184,burglary-business-by-force,burglary,6,611,cbd,2016-01-02 13:38:00,-104.995367,39.741134,burglary_business,...,0.0,NaN,0.0,NaT,NaN,0.0,NaN,0.0,NaT,NaN
93676,20164917,burglary-residence-by-force,burglary,3,322,virginia-village,2016-01-02 17:00:00,-104.904795,39.681983,burglary_residence,...,0.0,NaN,0.0,NaT,NaN,0.0,NaN,0.0,NaT,NaN
114356,20163610,burglary-residence-by-force,burglary,3,321,hilltop,2016-01-02 17:55:00,-104.928957,39.720677,burglary_residence,...,0.0,NaN,0.0,NaT,NaN,0.0,NaN,0.0,NaT,NaN


In [46]:
# TODO: remove?
# DEBUG: check if no previous event has a timestamp of a later date
for dist in proximity_distance:
    for idx, row in df.loc[df[f'day14_{dist}m_mintindex']>= 0].iterrows():
        date1 = row['timestamp']

        prev = df.loc[df['id']==row[f'day14_{dist}m_mintindex']]
        prevt = prev.timestamp.values[0]

        if prevt > date1:
            print('We have a problem')

In [47]:
# Write prepared file for further analisis
df.to_csv('../assets/data/prepared/denver_prepared.csv', index = False, header=True) 

----